In [ ]:
# Install the necessary library to interact with BigQuery
%pip install pandas-gbq
%pip install --upgrade google-genai

In [ ]:
import pandas_gbq
import bigframes.pandas as bf
import os

In [ ]:
# Use local computing power
sql = "SELECT * FROM demo_real_estate.property"
df_property = bf.read_gbq(sql)

# display(df_property.head())

In [ ]:
# Use local computing power
sql = "SELECT * FROM demo_real_estate.property_type"
df_type = bf.read_gbq(sql)

# display(df_type.head())

In [ ]:
# Merge
merged_df = df_property.merge(df_type, left_on='property_type_id', right_on='property_type_key', how='left')

# Filtr
filter_df = merged_df[merged_df['type_name'].str.lower() == 'byt']

# Final filter
final_df = filter_df[['property_id', 'description']].head(10)

display(final_df)

In [ ]:
from google import genai
from google.genai import types

def call_ai(description: str) -> str:
    client = genai.Client(vertexai=True, project="adcz-adoki-poc", location="global")

    system_instruction = "You will be provided with description of a property that is listed on a real-estate website. Only print the size of the property in square meters. If not provided print 'None'"

    contents = [
        types.Content(
            role="user",
            parts=[types.Part.from_text(text=str(description or ""))]
        )
    ]

    cfg = types.GenerateContentConfig(
        temperature=0,
        top_p=0.0,
        max_output_tokens=64,
        safety_settings=[
            types.SafetySetting(category="HARM_CATEGORY_HATE_SPEECH", threshold="OFF"),
            types.SafetySetting(category="HARM_CATEGORY_DANGEROUS_CONTENT", threshold="OFF"),
            types.SafetySetting(category="HARM_CATEGORY_SEXUALLY_EXPLICIT", threshold="OFF"),
            types.SafetySetting(category="HARM_CATEGORY_HARASSMENT", threshold="OFF"),
        ],
        system_instruction=[types.Part.from_text(text=system_instruction)],
    )

    resp = client.models.generate_content(
        model="gemini-2.5-flash-lite",
        contents=contents,
        config=cfg,
    )
    return (getattr(resp, "text", "") or "").strip()

In [ ]:
pdf = final_df.to_pandas()
pdf["flat_size"] = pdf["description"].astype(str).apply(call_ai)

import pandas_gbq
pandas_gbq.to_gbq(
    pdf,
    destination_table="demo_real_estate.property_ai",
    project_id="adcz-adoki-poc",
    if_exists="replace"
)